# Predicitions Demonstration

This notebook briefly workouts the details of getting a predicition pipeline that utilizes face detection, followed by mask classifaction. This demonstration will produce 4 images with classifications added which will be reviewed in the README.md

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import cv2
import os

These variables would be declared at the top of a prediction file or inside a configuration file

In [2]:
# Variable Constants #
ORIG_INPUT_DATASET = "mask-classifier-images"
BASE_PATH = "dataset"

# attached to path below
TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

# classes and color coding
CLASSES = ["Mask", "No_Mask"]
CLASS_COLORS = [(0,255,0),(0, 0, 255)]

# Path for Label encoder
LE_PATH = os.path.sep.join(["output", "le.cpickle"])
BASE_CSV_PATH = "output"

# model path
MODEL_PATH = os.path.sep.join(["output", "maskclassifier.model"])

BATCH_SIZE = 32

These function are defined for the main funciton, as well as the colors

In [13]:
# Default colors
COLOR_BLUE = (255, 0, 0)
COLOR_GREEN = (0, 255, 0)
COLOR_RED = (0, 0, 255)
COLOR_WHITE = (255, 255, 255)
COLOR_YELLOW = (0, 255, 255)

# Draw the predicted bounding box
def draw_predict(frame, conf, left, top, right, bottom, classidx):
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), CLASS_COLORS[classidx], 2)

    text = '{}: {:.2f}%'.format(CLASSES[classidx], conf * 100)

    # Display the label at the top of the bounding box
    label_size, base_line = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

    top = max(top, label_size[1])
    cv2.putText(frame, text, (left, top - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                CLASS_COLORS[classidx], 1)
    #print('drew something on output')
    
def refined_box(left, top, width, height):
    right = left + width
    bottom = top + height

    original_vert_height = bottom - top
    top = int(top + original_vert_height * 0.15)
    bottom = int(bottom - original_vert_height * 0.05)

    margin = ((bottom - top) - (right - left)) // 2
    left = left - margin if (bottom - top - right + left) % 2 == 0 else left - margin - 1

    right = right + margin

    return left, top, right, bottom

def crop_pic(frame, left, top, right, bottom):
    frame = frame[top:bottom+1, left: right+1]
    return frame

def maskPredict(model, image):
    image = cv2.resize(image, (224, 224))
    #forward pass the image through
    preds = model.predict(np.expand_dims(image,axis=0))[0]
    i = np.argmax(preds)

    return i

Test image preperation for models

In [4]:
#define path to image for testing
img_path = "facewithmask.jpg"

#make clone of image to draw on later. Extract resolution for scaling later
image = cv2.imread(img_path)
output = image.copy()
output = imutils.resize(output, width=400)
(h, w) = output.shape[:2]

# convert image's color, and create blob for face detection, create image copy to crop from later
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
blob4Face = cv2.dnn.blobFromImage(cv2.resize(image, (416,416)), 1/255.0, (416,416), crop=False)

# convert the image to a floats and subtract mean. This will be passed to mask detection
image = image.astype("float32")
mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
image -= mean

One run through example, to be turned into a function in the next cell following

In [5]:
#define path to image for testing
img_path = "facewithmask.jpg"

#make clone of image to draw on later. Extract resolution for scaling later
image = cv2.imread(img_path)
output = image.copy()
output = imutils.resize(output, width=400)
(h, w) = output.shape[:2]

# convert image's color, and create blob for face detection, create image copy to crop from later
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
blob4Face = cv2.dnn.blobFromImage(cv2.resize(image, (416,416)), 1/255.0, (416,416), crop=False)

# convert the image to a floats and subtract mean. This will be passed to mask detection
image = image.astype("float32")
mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
image -= mean

# load trained model from disk
print("[INFO] loading mask classifier model...")
mask_model = load_model(MODEL_PATH)

# load the model and get layer names for the output layers
print("[INFO] loading face detection model...")

net = cv2.dnn.readNetFromDarknet("models/yolov3-face.cfg", "models/yolov3-wider_16000.weights")
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# forward pass the net after inputing imgage. Out only includes unconnected output layers
net.setInput(blob4Face)
Outs = net.forward(ln)

#
confidences = []
boxes = []
final_boxes = []
for out in Outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * w)
            center_y = int(detection[1] * h)
            width = int(detection[2] * w)
            height = int(detection[3] * h)
            left = int(center_x - width/2)
            top = int(center_y - height/2)
            confidences.append(float(confidence))
            boxes.append([left, top, width, height])

# choose best choices out of overlapped boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

print(indices)
# this loop runs mask prediction and draws results to image
for i in indices:
    i = i[0]
    box = boxes[i]
    left = box[0]
    top = box[1]
    width = box[2]
    height = box[3]
    final_boxes.append(box)
    left, top, right, bottom = refined_box(left, top, width, height)
    crop = crop_pic(cv2.resize(image, (w, h)), left, top, right, bottom)
    cv2.imshow("Crop", crop)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    classidx = maskPredict(mask_model, crop)
    draw_predict(output, confidences[i], left, top, right, bottom, classidx)
    
cv2.imshow("Image", image)
cv2.imshow("Output", output)
cv2.imshow("Crop", crop)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] loading mask classifier model...
[INFO] loading face detection model...
[[4]]
drew something on output


In [19]:
def predicitonPipeline(img_path):
    #make clone of image to draw on later. Extract resolution for scaling later
    image = cv2.imread(img_path)
    output = image.copy()
    output = imutils.resize(output, width=400)
    (h, w) = output.shape[:2]

    # convert image's color, and create blob for face detection, create image copy to crop from later
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    blob4Face = cv2.dnn.blobFromImage(cv2.resize(image, (416,416)), 1/255.0, (416,416), crop=False)

    # convert the image to a floats and subtract mean. This will be passed to mask detection
    image = image.astype("float32")
    mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
    image -= mean

    # load trained model from disk
    print("[INFO] loading mask classifier model...")
    mask_model = load_model(MODEL_PATH)

    # load the model and get layer names for the output layers
    print("[INFO] loading face detection model...")

    net = cv2.dnn.readNetFromDarknet("models/yolov3-face.cfg", "models/yolov3-wider_16000.weights")
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # forward pass the net after inputing imgage. Out only includes unconnected output layers
    net.setInput(blob4Face)
    Outs = net.forward(ln)

    #
    confidences = []
    boxes = []
    final_boxes = []
    for out in Outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                width = int(detection[2] * w)
                height = int(detection[3] * h)
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # choose best choices out of overlapped boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # this loop runs mask prediction and draws results to image
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        final_boxes.append(box)
        left, top, right, bottom = refined_box(left, top, width, height)
        crop = crop_pic(cv2.resize(image, (w, h)), left, top, right, bottom)
        classidx = maskPredict(mask_model, crop)
        draw_predict(output, confidences[i], left, top, right, bottom, classidx)

    cv2.imshow("Image", image)
    cv2.imshow("Output", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return output

Final cell runs algorithm on demo images and places them in a folder.

In [21]:
image_list = ['demo_image_1','demo_image_2','demo_image_3','demo_image_4']


for img in image_list:
    img_path = img + '.jpg'
    output = predicitonPipeline(img_path)
    processed_path = 'images/' + img + '(processed).jpg'
    cv2.imwrite(processed_path ,output)

[INFO] loading mask classifier model...
[INFO] loading face detection model...
[INFO] loading mask classifier model...
[INFO] loading face detection model...
[INFO] loading mask classifier model...
[INFO] loading face detection model...
[INFO] loading mask classifier model...
[INFO] loading face detection model...
